# 챕터 1. 밴디트 문제

1.1 머신러닝 분류와 강화 학습  
>1.1.1 지도 학습  
1.1.2 비지도 학습  
1.1.3 강화 학습

1.2 밴디트 문제 와 표기법
>1.2.1 밴디트 문제란?  
  환경, 에이전트, 행동, 보상  
>1.2.2 좋은 슬롯머신이란?  
  확률분포표, 기댓값  
>1.2.3 수식으로 표현하기   
보상(Reward) = R  
확률변수 = 값이 확률적으로 결정되는 변수  
행동(Action) = A  
기댓값(Expectaion) = E, $\mathop{\mathbb{E}}$  
보상 R의 기댓값 = $\mathbb{E}\left[ R \right]$   
t번째에 얻는 보상 R의 기댓값 = $\mathbb{E}\left[ R_{t} \right]$  
행동 A로 인한 보상 기댓값 = $\mathbb{E}\left[ R| A \right]$  
보상에 대한 기댓값 = 행동가치 Q, q  
행동 A에 대한 행동가치 $q \left( A \right) = \mathbb{E} \left[ R|A \right]$  
행동가치 Q 를 소문자로 적을 때는 실제 행동가치, 대문자 Q로 적을 때는 추정 행동가치를 의미

    

1.3 밴티드 알고리즘
> 슬롯머신의 가치를 모르고, 추정해나가야 하는 상황.  
> 1.3.1 가치 추정 방법  
표본 평균  
> 1.3.2 평균을 구하는 코드  
증분구현  
> 1.3.3 플레이어의 정책  
탐욕 정책(Greedy policy)  
활용 exploitation  
탐색 exploration  
활용과 탐색의 균형이 강화학습의 핵심. 기본적인 알고리즘은 $\epsilon$-탐욕 정책, 엡실론 그리디 정책  
$\epsilon$-탐욕 정책, 엡실론 그리디 정책 : $\epsilon$ = 0.1  이라면, 10% 확률로 탐색을 하고, 90%는 활용을 한다. 탐색에서는 무작위 행동을 한다. 

1.4 밴디트 알고리즘 구현  
> 1.4.1 슬롯머신 구현

In [21]:
import numpy as np

class Bandit:
    def __init__(self, arms=10):
        self.rates = np.random.rand(arms) # 슬롯머신들의 승률 설정 

    def play(self, arm):
        rate = self.rates[arm]
        if rate > np.random.rand():
            return 1
        return 0

    def show(self):
        print(self.rates)

In [23]:
bandit = Bandit()
bandit.show()

[0.36928192 0.8913941  0.14736506 0.43569083 0.42526147 0.50362806
 0.50826298 0.706669   0.14693915 0.75988704]


In [79]:
_sum = 0
for i in range(3):
    _ = bandit.play(0)
    _sum+=_
    print(_)
# _sum

0
0
0


1.4.2 에이전트 구현

In [85]:
bandit = Bandit()
Q = 0

In [86]:
bandit.show()

[0.14231588 0.62336533 0.90899853 0.52419338 0.26677634 0.84189895
 0.14967062 0.073204   0.99540357 0.04730611]


In [88]:
for n in range(1, 11):
    reward = bandit.play(0)
    Q += (reward - Q) / n
    print(Q)

0.0
0.0
0.0
0.0
0.0
0.0
0.14285714285714285
0.125
0.2222222222222222
0.3


In [ ]:
bandit = Bandit()
Qs = np.zeros(10) # 각 슬롯머신의 가치 추정치
ns = np.zeros(10) # 각 슬롯머신의 플레이 횟수

for n in range(10):
    action = np.random.randint(0, 10) # 임의의 슬롯머신 선택
    